# MLiP Groupby with pandas
Kurs Maschinelles Lernen in der Produktion

#### Zielstellung:

In manchen Datensätzen existieren je Sample mehrere Messwerte für ein Feature (siehe df_data_raw, Output 1). 

Um die im Kurs gelernten Verfahren anwenden zu können, ist ein Datensatz mit nur einem Messwert je Sample und Feature nötig (siehe df_data_should, Output 2). 

Hierfür wird die Groupby-Funktion von pandas verwendet, um die mehreren Messwerte eines Samples und Feature zu einem Wert zusammenzufassen.

In [ ]:
# Bibliotheken importieren
import numpy as np
import pandas as pd

In [ ]:

df_data_raw = pd.DataFrame({'sample_ID': [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4], 
                        'acc_y': [0.1, 0.01, 0.2, 3.1, 0.2, 0.1, 0.1, 0.21, 0.05, 0.1, 2.4, 3.2],
                        'time_secs': [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2],
                        'part_ID': [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1],
                        'class': ['ok', 'ok', 'ok', 'dam', 'dam', 'dam', 'ok', 'ok', 'ok', 'dam', 'dam', 'dam']})
print('Der rohe Datensatz mit 3 Messungen für die y-Beschleunigung (je 3 Zeitpunkten (0, 1, 2)) für jedes Sample.')
df_data_raw

In [ ]:
df_data_should = pd.DataFrame({'sample_ID': [1, 2, 3, 4], 
                               'acc_y': ['?', '?', '?', '?'],
                               'class': ['ok', 'dam', 'ok', 'dam']})
print('Gewünschte Struktur des Datensatzes')
df_data_should

### 1. Erstellen der Spalten Sample (ID) und der Zielvariable
Es wird als erstes die Spalte ID und Klasse für den neuen Datensatz erstellt. 

1. Benötigte Spalten aus Rohdaten:
    - Spalten ID ('sample_ID') und Klasse ('class')


2. Zusammenfassen mittels .groupby
    - Nun wird mittels .groupby('sample_ID') der Datensatz so zusammengefasst, dass alle Zeilen mit gleicher sample_ID zu einer Zeile zusammengefasst werden. 
    

3. Operation, wie die Messwerte zu einem Wert zusammengefasst werden sollen
    - Mit .first() wird festgelegt, dass von allen Zeilen mit gleicher sample_ID nur die erste (.first()) verwendet wird.
    
    
4. Rückumwandlung in DataFrame-Object
    - Mit .reset_index() wird das Gouped-Object in ein DataFrame-Object transformiert.

In [ ]:
df_new_id = df_data_raw[['sample_ID', 'class']].groupby('sample_ID').first().reset_index()
df_new_id

### 2. Erstellen eines 1. Features
Es wird ein Feature für den neuen Datensatz erstellt. 

1. Benötigte Spalten aus Rohdaten:
    - Spalten ID und Feature
    

2. Zusammenfassen mittels .groupby
    - Nun wird mittels .groupby('sample_ID') der Datensatz so zusammengefasst, dass alle Zeilen mit gleicher sample_ID zu einer Zeile zusammengefasst werden. 
    

3. Aggregatfunktion, wie die Messwerte zu einem Wert zusammengefasst werden sollen
    - Mit .mean() wird festgelegt, dass von allen Zeilen mit gleicher sample_ID der Mittelwert als neuer Wert verwendet wird.
    
    
4. Rückumwandlung in DataFrame-Object
    - Mit .reset_index() wird das Gouped-Object in ein DataFrame-Object transformiert.

In [ ]:
df_new_mean = df_data_raw[['sample_ID', 'acc_y']].groupby('sample_ID').mean().reset_index()
df_new_mean = df_new_mean.rename(columns={'acc_y': 'mean_acc_y'})
df_new_mean

### 3. Erstellen eines 2. Feautures
Es wird ein Feature für den neuen Datensatz erstellt. 

1. Benötigte Spalten aus Rohdaten:
    - Spalten ID und Feature
    

2. Zusammenfassen mittels .groupby
    - Nun wird mittels .groupby('sample_ID') der Datensatz so zusammengefasst, dass alle Zeilen mit gleicher sample_ID zu einer Zeile zusammengefasst werden. 
    

3. Aggregatfunktion, wie die Messwerte zu einem Wert zusammengefasst werden sollen
    - Mit .max() wird festgelegt, dass von allen Zeilen mit gleicher sample_ID der Maximalwert als neuer Wert verwendet wird.
    
    
4. Rückumwandlung in DataFrame-Object
    - Mit .reset_index() wird das Gouped-Object in ein DataFrame-Object transformiert.

In [ ]:
df_new_max = df_data_raw[['sample_ID', 'acc_y']].groupby('sample_ID').max().reset_index()
df_new_max = df_new_max.rename(columns={'acc_y': 'max_acc_y'})
df_new_max

### 4. Mergen zu Datensatz

In [ ]:
df_new = pd.merge(df_new_id, df_new_mean, on='sample_ID', how='inner')
df_new

In [ ]:
df_new = pd.merge(df_new, df_new_max, on='sample_ID', how='inner')
df_new

Der Datensatz ist hiermit fertig. Es wurden sogar für das Signal (acc_y) zwei Features (mean und max) gebildet. Der erstellte DataFrame kann nun für das Maschinelle Lernen verwendet werden.

__Achtung__: Hier geschieht eine Informationsreduktion, da die 3 Messwerte je Sample zu einem zusammengefasst werden (Hier einemal zum Mittelwert und einmal zum Maximalwert).

Die erstellten Features müssen den Zusammenhang der Problemstellung abbiden/enthalten um ein gutes Modell zu trainieren.

### Einige mögliche interene Funktionen zur Zusammenfassung von Werten:

- min()
- max()
- mean()
- median()


- std()
- var()
- mad()


- sum()
- prod()
- count()


- first()
- last()

__Hinweis__  
Es können auch eigene Funktionen definiert werden und als Aggregatfunktion genutzt werden.

## Anwendung von mehreren Aggregatfunktionen bei groupby

Anstatt mehrere Features jeweils einzeln zu kreieren und die Ergebnisse anschließend zu mergen, können mehrere Aggregatfunktionen gleichzeitig und Spaltenweise übergeben werden.  
Dies erfolgt über die Anwendung des Befehls ".agg" nach dem groupby.  
Hier wird als Dictionary für jede Spalte die entsprechenden anzuwendenen Funktion bzw. Funktionen angegeben.  

{'zu_aggregierende_Spalte' : Aggregatfunktion,  
'zu_aggregierende_Spalte2' : [Aggregatfunktion1, Aggregatfunktion2, ...]  
}

Zusätzlich soll noch eine eigene Aggregatfunktion definiert werden. 

In [ ]:
#Definiton einer eigenen Aggregatfunktion - Beispiel RMS - Effektivwert
# Achtung, eigene Aggregatfunktionen bekommen als Argument eine pandas series

def RMS(series):
    """
    Calculates root mean square (as measurement of energy level).
    """
    series = series.to_numpy()
    return np.sqrt(np.mean(np.square(series))) if len(series) > 0 else np.NaN

Anwenden von mehreren Aggregatfunktionen bei groupby mittels dem .agg Befehls.  
__Hinweis__
* Es empfiehlt sich dem groupby Befehl den Parameter "as_index = False" übergeben, um eine neue Indexspalte generieren zu lassen. 

In [ ]:
df_grouped = df_data_raw.groupby('sample_ID', as_index = False).agg({'acc_y': ['mean', 'max', RMS], 
                                                                     'class': 'first'})
df_grouped.head(5)

__Achtung__  
Die Verwendung von mehreren Aggregatfunktionen führt zu einen mehrschichtigen Tabellenkopf:

__Lösungsvorschlag__  
Der automatisch einzeilige Kopfzeile erzeugt

In [ ]:
# Using list comprehension, a string join and a filter, we can create better names for the columns:
df_grouped.columns = ["_".join(filter(None, x)) for x in df_grouped.columns]
df_grouped.head(5)

Mittels dem Befehl ".rename()" können noch grundsätzlich Spalten umbenannt werden

In [ ]:
# Umbennennen der class_first Spalte in class
df_grouped = df_grouped.rename(columns = {'class_first': 'class'})
df_grouped